In [1]:
import pandas as pd
import numpy as np
from statistics import mode

In [2]:
# read data
concert_name = "concerts_2014-15"
account = pd.read_csv("data/account.csv", encoding="latin-1", delimiter=",")
concerts = pd.read_csv("data/concerts.csv", delimiter=",")
concerts_2014_15 = pd.read_csv(f"data/{concert_name}.csv", delimiter=",")
subscriptions = pd.read_csv("data/subscriptions.csv", delimiter=",")
train = pd.read_csv("data/train.csv", delimiter=",")
test = pd.read_csv("data/test.csv", delimiter=",")
tickets_all = pd.read_csv("data/tickets_all.csv", delimiter=",")
zipcodes = pd.read_csv("data/zipcodes.csv", delimiter=",")

In [3]:
def rename_columns(df):
    df.rename(columns={col: col.replace('.', '_') for col in df.columns}, inplace=True)

# Apply the function to each DataFrame
rename_columns(account)
rename_columns(train)
rename_columns(test)
rename_columns(tickets_all)
rename_columns(subscriptions)
rename_columns(concerts)
rename_columns(concerts_2014_15)
rename_columns(zipcodes)

In [4]:
# columns to delete that won't have much
all_columns = {}
all_columns['account'] = list(account.columns)
all_columns['concerts'] = list(concerts.columns)
all_columns['concerts_2014_15'] = list(concerts_2014_15.columns)
all_columns['subscriptions'] = list(subscriptions.columns)
all_columns['train'] = list(train.columns)
all_columns['test'] = list(test.columns)
all_columns['tickets_all'] = list(tickets_all.columns)
all_columns['zipcodes'] = list(zipcodes.columns)
all_columns

{'account': ['account_id',
  'shipping_zip_code',
  'billing_zip_code',
  'shipping_city',
  'billing_city',
  'relationship',
  'amount_donated_2013',
  'amount_donated_lifetime',
  'no_donations_lifetime',
  'first_donated'],
 'concerts': ['season', 'concert_name', 'set', 'who', 'what', 'location'],
 'concerts_2014_15': ['season', 'concert_name', 'set', 'who', 'what'],
 'subscriptions': ['account_id',
  'season',
  'package',
  'no_seats',
  'location',
  'section',
  'price_level',
  'subscription_tier',
  'multiple_subs'],
 'train': ['account_id', 'label'],
 'test': ['ID'],
 'tickets_all': ['account_id',
  'price_level',
  'no_seats',
  'marketing_source',
  'season',
  'location',
  'set',
  'multiple_tickets'],
 'zipcodes': ['Zipcode',
  'ZipCodeType',
  'City',
  'State',
  'LocationType',
  'Lat',
  'Long',
  'Location',
  'Decommisioned',
  'TaxReturnsFiled',
  'EstimatedPopulation',
  'TotalWages']}

In [5]:
print(train.shape, test.shape, account.shape, subscriptions.shape, concerts.shape, concerts_2014_15.shape, zipcodes.shape, tickets_all.shape)

(6941, 2) (2975, 1) (19833, 10) (28627, 9) (103, 6) (6, 5) (42522, 12) (2808, 8)


### Rename test ID name to account_id

In [6]:
test.rename(columns={'ID': 'account_id'}, inplace=True)
test.head(1)

,account_id
0,001i000000NuQ6Y


### Merge Datasets

In [7]:
train_copy = train.copy()
test_copy = test.copy()

In [8]:
train_account = pd.merge(account, train_copy, on="account_id", how="inner")
test_account = pd.merge(account, test_copy, on="account_id", how="inner")

In [9]:
print(train_account.shape, test_account.shape)

(6941, 11) (2975, 10)


In [10]:
def preprocess_price_level(df):
    df['price_level'] = df['price_level'].astype(str)
    df['price_level'] = pd.Categorical(df['price_level'], ordered=True, categories=df['price_level'].unique())
    df['price_level'] = df['price_level'].cat.codes
    
def preprocess_subs_tier(df):
    df['subscription_tier'] = df['subscription_tier'].astype(str)
    df['subscription_tier'] = pd.Categorical(df['subscription_tier'], ordered=True, categories=df['subscription_tier'].unique())
    df['subscription_tier'] = df['subscription_tier'].cat.codes
    
preprocess_price_level(tickets_all)
preprocess_price_level(subscriptions)
preprocess_subs_tier(subscriptions)

In [11]:
subscriptions.shape, concerts.shape, concerts_2014_15.shape, tickets_all.shape

((28627, 9), (103, 6), (6, 5), (2808, 8))

In [12]:
all_concerts = pd.concat([concerts, concerts_2014_15], axis=0)
all_concerts.shape

(109, 6)

In [13]:
subscription_concerts = pd.merge(subscriptions, all_concerts.add_suffix('_concerts'), left_on="season", right_on="season_concerts", how='left')
tickets_concerts = pd.merge(tickets_all, all_concerts.add_suffix('_concerts'), left_on="season", right_on="season_concerts", how='left')
subscription_concerts.shape, tickets_concerts.shape

((131264, 15), (68899, 14))

In [14]:
subscription_concerts.columns, tickets_all.columns

(Index(['account_id', 'season', 'package', 'no_seats', 'location', 'section',
        'price_level', 'subscription_tier', 'multiple_subs', 'season_concerts',
        'concert_name_concerts', 'set_concerts', 'who_concerts',
        'what_concerts', 'location_concerts'],
       dtype='object'),
 Index(['account_id', 'price_level', 'no_seats', 'marketing_source', 'season',
        'location', 'set', 'multiple_tickets'],
       dtype='object'))

In [15]:
subscription_concerts.drop(columns=['season_concerts'], inplace=True)
tickets_concerts.drop(columns=['season_concerts'], inplace=True)

In [16]:
subscription_concerts.columns

Index(['account_id', 'season', 'package', 'no_seats', 'location', 'section',
       'price_level', 'subscription_tier', 'multiple_subs',
       'concert_name_concerts', 'set_concerts', 'who_concerts',
       'what_concerts', 'location_concerts'],
      dtype='object')

In [17]:
tickets_concerts.columns

Index(['account_id', 'price_level', 'no_seats', 'marketing_source', 'season',
       'location', 'set', 'multiple_tickets', 'concert_name_concerts',
       'set_concerts', 'who_concerts', 'what_concerts', 'location_concerts'],
      dtype='object')

In [18]:
# Aggregate ticket and subscription data
tickets_concerts = tickets_concerts.groupby('account_id').agg({'no_seats': 'sum', 'price_level': lambda x: mode(x)}).reset_index()
subscription_concerts = subscription_concerts.groupby('account_id').agg({'no_seats': 'sum', 'price_level': lambda x: mode(x), 'subscription_tier': lambda x: mode(x)}).reset_index()

In [19]:
# Merge ticket and subscription data
train_account_tickets = pd.merge(train_account, tickets_concerts.add_suffix('_tickets'), left_on="account_id", right_on="account_id_tickets", how='left')
train_account_subs = pd.merge(train_account_tickets, subscription_concerts.add_suffix('_subs'), left_on="account_id", right_on="account_id_subs", how='left')

test_account_tickets = pd.merge(test_account, tickets_concerts.add_suffix('_tickets'), left_on="account_id", right_on="account_id_tickets", how='left')
test_account_subs = pd.merge(test_account_tickets, subscription_concerts.add_suffix('_subs'), left_on="account_id", right_on="account_id_subs", how='left')

In [20]:
test_account_subs.shape, train_account_subs.shape 

((2975, 17), (6941, 18))

In [21]:
train_account_subs.columns

Index(['account_id', 'shipping_zip_code', 'billing_zip_code', 'shipping_city',
       'billing_city', 'relationship', 'amount_donated_2013',
       'amount_donated_lifetime', 'no_donations_lifetime', 'first_donated',
       'label', 'account_id_tickets', 'no_seats_tickets',
       'price_level_tickets', 'account_id_subs', 'no_seats_subs',
       'price_level_subs', 'subscription_tier_subs'],
      dtype='object')

In [22]:
train_account_subs = train_account_subs.drop(['account_id_subs', 'account_id_tickets'], axis=1)

In [23]:
test_account_subs = test_account_subs.drop(['account_id_subs', 'account_id_tickets'], axis=1)

In [24]:
train_account_subs.dtypes

account_id                  object
shipping_zip_code           object
billing_zip_code            object
shipping_city               object
billing_city                object
relationship                object
amount_donated_2013        float64
amount_donated_lifetime    float64
no_donations_lifetime        int64
first_donated               object
label                        int64
no_seats_tickets           float64
price_level_tickets        float64
no_seats_subs              float64
price_level_subs           float64
subscription_tier_subs     float64
dtype: object

In [25]:
# # years since donation 2013
train_account_subs['first_donated'] = pd.to_datetime(train_account_subs['first_donated'])
test_account_subs['first_donated'] = pd.to_datetime(test_account_subs['first_donated'])

train_account_subs['years_since_2013'] = 2013 - train_account_subs['first_donated'].dt.year
test_account_subs['years_since_2013'] = 2013 - test_account_subs['first_donated'].dt.year

train_account_subs['years_since_2013'] = pd.to_numeric(train_account_subs['years_since_2013'], downcast='integer')
test_account_subs['years_since_2013'] = pd.to_numeric(test_account_subs['years_since_2013'], downcast='integer')

In [26]:
test_account_subs.head()

,account_id,shipping_zip_code,billing_zip_code,shipping_city,billing_city,relationship,amount_donated_2013,amount_donated_lifetime,no_donations_lifetime,first_donated,no_seats_tickets,price_level_tickets,no_seats_subs,price_level_subs,subscription_tier_subs,years_since_2013
0,001i000000LhyPF,NaN,94610,NaN,Oakland,NaN,1500.0,28435.00,28,1986-07-04,NaN,NaN,233.0,5.0,1.0,27.0
1,001i000000LhyPG,NaN,94024,NaN,Los Altos,NaN,300.0,4969.00,16,1997-04-07,NaN,NaN,228.0,5.0,1.0,16.0
2,001i000000LhyPP,NaN,94618,NaN,Oakland,NaN,0.0,4047.96,21,1998-12-02,NaN,NaN,227.0,0.0,1.0,15.0
3,001i000000LhyPb,NaN,94941,NaN,Mill Valley,NaN,0.0,5600.00,8,2007-05-13,NaN,NaN,212.0,5.0,0.0,6.0
4,001i000000LhyPg,NaN,94705,NaN,Berkeley,NaN,150.0,3678.00,35,1988-07-04,NaN,NaN,237.0,0.0,1.0,25.0


In [27]:
train_account_subs['zip_reduced'] = train_account_subs['billing_zip_code'].str[:2]
train_account_subs['zip_reduced'] = np.where(pd.to_numeric(train_account_subs['zip_reduced'], errors='coerce').isna(), 'nan', train_account_subs['zip_reduced'])
train_account_subs['zip_reduced'] = np.where(np.logical_and(train_account_subs['zip_reduced']=='nan',
                                                ~train_account_subs['billing_zip_code'].isna()),
                                 'foreign',
                                 train_account_subs['zip_reduced'])
train_account_subs['zip_reduced'].unique()

test_account_subs['zip_reduced'] = test_account_subs['billing_zip_code'].str[:2]
test_account_subs['zip_reduced'] = np.where(pd.to_numeric(test_account_subs['zip_reduced'], errors='coerce').isna(), 'nan', test_account_subs['zip_reduced'])
test_account_subs['zip_reduced'] = np.where(np.logical_and(test_account_subs['zip_reduced']=='nan',
                                                ~test_account_subs['billing_zip_code'].isna()),
                                 'foreign',
                                 test_account_subs['zip_reduced'])
test_account_subs['zip_reduced'].unique()

array(['94', '95', '23', 'nan', '50', '44', '10', '29', '61', '90',
       'foreign', '12', '68', '20', '04', '11', '27', '76', '92', '19',
       '60', '75', '18', '91', '85', '21', '96', '80', '97', '98', '43',
       '87', '93', '02', '83', '24', '84', '40', '77', '15', '32', '01',
       '17', '53', '26', '34', '48', '16', '59', '78', '66', '89', '65',
       '42', '70', '22', '13', '28', '47', '71', '08'], dtype=object)

In [28]:
train_account_subs['zip_matched'] = np.where(train_account_subs['shipping_zip_code'].isna(), '0', 1)
train_account_subs['zip_matched'] = np.where(np.logical_and(train_account_subs['zip_matched']==1,
                                            train_account_subs['shipping_zip_code'] == train_account_subs['billing_zip_code']),
                                 '1',
                                 '2')

test_account_subs['zip_matched'] = np.where(test_account_subs['shipping_zip_code'].isna(), '0', 1)
test_account_subs['zip_matched'] = np.where(np.logical_and(test_account_subs['zip_matched']==1,
                                            test_account_subs['shipping_zip_code'] == test_account_subs['billing_zip_code']),
                                 '1',
                                 '2')

In [29]:
train_account_subs = train_account_subs.drop(['shipping_zip_code', 'shipping_city', 'relationship', 'billing_zip_code', 'billing_city', 'first_donated'], axis=1)
test_account_subs = test_account_subs.drop(['shipping_zip_code', 'shipping_city', 'relationship', 'billing_zip_code', 'billing_city', 'first_donated'], axis=1)


In [30]:
train_account_subs.shape

(6941, 13)

In [31]:
train_account_subs.to_csv('data/merged_train.csv', index=False)
test_account_subs.to_csv('data/merged_test.csv', index=False)